In [1]:
import numpy as np
import pandas as pd


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, cross_val_predict


from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import FeatureUnion


from sklearn.externals import joblib

In [2]:
df_train = pd.read_csv('products_sentiment_train.tsv', sep='\t', names=['text','label'], header=None)

In [3]:
def cross_validate_rezult(cv_rez, pipe=True):
    if pipe:
        for step in cv_rez['estimator'][0].steps:
            if step[0] == 'est':
                est = type(step[1]).__name__
    else:
        est = type(cv_rez['estimator'][0]).__name__
    return [
        est,
        cv_rez['test_score'].mean(),
        cv_rez['test_score'].std(),
        len(cv_rez['estimator']),
        cv_rez['fit_time'].mean()            
    ]

In [14]:
%%time
rez = []
# my_token= '(?u)\\b\\w+\\b|[\!\?\.,\)-]'
my_token= '(?u)(\\b[A-Za-z][A-Za-z\!\?\.,\)-]+\\b)'

# for est in [SVC(C=1.265, kernel='linear')]:    
for est in [SVC(kernel='linear'),  LogisticRegression( solver='lbfgs' ), RidgeClassifier(), 
        MultinomialNB(), BernoulliNB(), ComplementNB(),
        RandomForestClassifier(),  GradientBoostingClassifier(), 
        SGDClassifier(), LinearSVC()]:  

#     pipe = Pipeline([("cv", TfidfVectorizer(token_pattern=my_token, ngram_range=(1, 3), analyzer='word' ) ),
#                  ("est", est)])
    
    cv_union = FeatureUnion(
        [
            ("cv_word", TfidfVectorizer(token_pattern=my_token, ngram_range=(2, 4), analyzer='word')),
            ("cv_char", TfidfVectorizer(token_pattern=my_token, ngram_range=(1, 5), analyzer='char_wb'))
        ]
    )
    pipe = Pipeline(
        [
            ("cv", cv_union ),
            ("est", est )
        ]
    )    
     
    cv_rez = cross_validate(pipe, df_train.text.values, df_train.label.values, cv=10, n_jobs=4,
                            return_estimator=True, scoring='accuracy' )
    rez.append(cross_validate_rezult(cv_rez))

Wall time: 6min 5s


In [15]:
rez_df = pd.DataFrame(rez, columns=['Алгоритм классификации', 
                                   'accuracy', 
                                   'Дисп. accuracy',
                                   'Количество фолдов', 'Время обучения фолда'])
rez_df.round(decimals=4).sort_values(by=['accuracy'])

,Алгоритм классификации,accuracy,Дисп. accuracy,Количество фолдов,Время обучения фолда
3,MultinomialNB,0.6395,0.0033,10,1.9115
5,ComplementNB,0.6490,0.0073,10,2.5656
4,BernoulliNB,0.7030,0.0156,10,2.4323
6,RandomForestClassifier,0.7130,0.0254,10,2.3479
7,GradientBoostingClassifier,0.7330,0.0153,10,40.8863
1,LogisticRegression,0.7740,0.0158,10,4.5643
8,SGDClassifier,0.7914,0.0227,10,2.0031
0,SVC,0.8015,0.0178,10,11.6513
9,LinearSVC,0.8015,0.0180,10,2.1234
2,RidgeClassifier,0.8020,0.0170,10,2.8458


In [4]:
%%time
# my_token= '(?u)\\b\\w\\w+\\b|[\!\?\.,\)-]'
my_token= '(?u)(\\b[A-Za-z][A-Za-z\!\?\.,\)-]+\\b)'
# my_token= '(?u)(\\b\\w[\\w\!\?\.,\)-]+\\b)'
# token_pattern=my_token,
cv_union = FeatureUnion(
    [
        ("cv_word", TfidfVectorizer(token_pattern=my_token, analyzer='word')),
        ("cv_char", TfidfVectorizer(token_pattern=my_token, analyzer='char_wb'))
    ]
)
pipe = Pipeline(
    [
        ("cv", cv_union ),
        ("est", SVC(kernel='linear'))
    ]
)


param_grid = {
#     'cv__cv_word__ngram_range': [(2, 5), (1, 3), (2, 6), (2, 4)],
    'cv__cv_word__ngram_range': [ (2, 4) ],
    'cv__cv_char__ngram_range': [ (1, 5)]
#     'cv__min_df': (0., 0.01),
#               'cv__max_df': ( 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22),
#               'cv__max_features': [200, 500, 1000],
#         'cv__norm': ['l2', 'l1', None],
#         'cv__ngram_range': [(2, 2), (2, 3), (1, 3), (2, 4)],
#               'est__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
#                 'est__penalty' : ['none', 'l2'],
#                'est__C' : np.logspace(-4, 4, 20),   
             } 


grid = GridSearchCV(pipe, param_grid, cv=10, n_jobs=10, scoring='accuracy')
grid3 = grid.fit(df_train.text.values, df_train.label.values)

Wall time: 41.6 s


In [5]:
print('best_score_', grid3.best_score_ )
grid3.best_estimator_.steps

best_score_ 0.8015


[('cv', FeatureUnion(n_jobs=None,
               transformer_list=[('cv_word',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content', lowercase=True,
                                                  max_df=1.0, max_features=None,
                                                  min_df=1, ngram_range=(2, 4),
                                                  norm='l2', preprocessor=None,
                                                  smooth_idf=True,
                                                  stop_words=None,
                                                  strip_accents=None,
                                                  sublinear_tf=Fals...
                   

In [105]:
# grid3.best_estimator_.steps[0][1].vocabulary_
grid3.best_estimator_.steps[0][1].transformer_list

[('cv_word',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.float64'>, encoding='utf-8',
                  input='content', lowercase=True, max_df=1.0, max_features=None,
                  min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                  smooth_idf=True, stop_words=None, strip_accents=None,
                  sublinear_tf=False,
                  token_pattern='(?u)(\\b\\w[\\w\\!\\?\\.,\\)-]+\\b)',
                  tokenizer=None, use_idf=True, vocabulary=None)),
 ('cv_char',
  TfidfVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
                  dtype=<class 'numpy.float64'>, encoding='utf-8',
                  input='content', lowercase=True, max_df=1.0, max_features=None,
                  min_df=1, ngram_range=(3, 6), norm='l2', preprocessor=None,
                  smooth_idf=True, stop_words=None, strip_accents=None,
                  sublinear_tf=False,
               

In [102]:
grid3.best_estimator_.steps[0][1].transformer_list[0][1].vocabulary_

{'take around': 86732,
 'around 10,000': 10455,
 '10,000 640x480': 95,
 '640x480 pictures': 1024,
 'take around 10,000': 86733,
 'around 10,000 640x480': 10456,
 '10,000 640x480 pictures': 96,
 'take around 10,000 640x480': 86734,
 'around 10,000 640x480 pictures': 10457,
 'take around 10,000 640x480 pictures': 86735,
 'downloaded trial': 27156,
 'trial version': 104003,
 'version of': 106968,
 'of computer': 64178,
 'computer associates': 21916,
 'associates ez': 11463,
 'ez firewall': 30438,
 'firewall and': 32260,
 'and antivirus': 5122,
 'antivirus and': 8881,
 'and fell': 5850,
 'fell in': 31504,
 'in love': 43097,
 'love with': 55417,
 'with computer': 112047,
 'computer security': 21978,
 'security system': 79876,
 'system all': 86479,
 'all over': 3183,
 'over again': 69393,
 'downloaded trial version': 27157,
 'trial version of': 104004,
 'version of computer': 106969,
 'of computer associates': 64183,
 'computer associates ez': 21917,
 'associates ez firewall': 11464,
 'ez fi

0.8015 получено на cv=10 (12 минут) с параметрами  
* ngram_range=(1, 5), analyzer='char_wb'
* ngram_range=(2, 4), analyzer='word'
* my_token= '(?u)(\\b[A-Za-z][A-Za-z\!\?\.,\)-]+\\b)'

Почему в словаре есть цифры. При маске по умолчалнию '(?u)\\b\\w\\w+\\b'  
(?u) - означает применение UNICODE (Make the \w, \W, \b, \B, \d, \D, \s and \S sequences dependent on the Unicode character properties database. Also enables non-ASCII matching for IGNORECASE.)  
\w - Соответствует любому цифробуквенному символу включая нижнее подчеркивание. Эквивалентен [A-Za-z0-9_].



In [28]:
%%time
my_token= '(?u)(\\b[A-Za-z][A-Za-z\!\?\.,\)-]+\\b)'

cv_union = FeatureUnion(
    [
        ("cv_word", TfidfVectorizer(token_pattern=my_token, ngram_range=(2, 4), analyzer='word')),
        ("cv_char", TfidfVectorizer(token_pattern=my_token, ngram_range=(1, 5), analyzer='char_wb'))
    ]
)
pipe = Pipeline(
    [
        ("cv", cv_union ),
        ("est", SVC(kernel='linear', probability = True))
    ]
)


# pipe = Pipeline([("cv", TfidfVectorizer(token_pattern=my_token, ngram_range=(1, 3), analyzer='word' ) ),
#              ("est", SVC(kernel='linear', probability = True))])



cv_rez = cross_validate(pipe, df_train.text.values, df_train.label.values, cv=10, n_jobs=4,
                        return_estimator=True, scoring='accuracy' )

In [29]:
cv_rez['test_score'].mean()

0.8014534863371583

In [30]:
pipe.fit(df_train.text.values, df_train.label.values)

Pipeline(memory=None,
     steps=[('cv', FeatureUnion(n_jobs=None,
       transformer_list=[('cv_word', TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngra...ar', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [18]:
vectorized = pipe.steps[0][1].transform(['this is very good'])

In [21]:
pipe.steps[1][1].predict_proba(vectorized)[0].max()

0.999985041752358

In [22]:
joblib.dump( pipe.steps[0][1], 'BigramUnprocessedVectorizer.pkl')

['BigramUnprocessedVectorizer.pkl']

In [23]:
joblib.dump(pipe.steps[1][1], 'DefaultLogisticBigramUnprocessedTextSentiment.pkl')

['DefaultLogisticBigramUnprocessedTextSentiment.pkl']

In [31]:
joblib.dump(pipe, 'test.pkl')

['test.pkl']

In [32]:
unload_test = joblib.load('test.pkl')

In [33]:
unload_test.steps

[('cv', FeatureUnion(n_jobs=None,
         transformer_list=[('cv_word', TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
          dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(2, 4), norm=u'l2', preprocessor=None, ...u)(\\b[A-Za-z][A-Za-z\\!\\?\\.,\\)-]+\\b)',
          tokenizer=None, use_idf=True, vocabulary=None))],
         transformer_weights=None)),
 ('est', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False))]

In [35]:
unload_test.predict_proba(['this is good'])

array([[1.61277265e-05, 9.99983872e-01]])